In [1]:
import pandas as pd
import numpy as np
#from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from featuretools.primitives import AggregationPrimitive, make_agg_primitive
from datetime import datetime, timedelta
import re
import featuretools as ft

In [2]:
data = pd.read_csv("\\Users\\JoonH\\total_feature_matrix_spec.csv")

In [3]:
#Already saved the data to csv
#train.to_csv("\\Users\\JoonH\\JoonH_credit_train.csv")
#train.to_csv("\\Users\\JoonH\\JoonH_credit_test.csv")

In [4]:
train = data[data['set'] == 'train']
test = data[data['set'] == 'test']

In [5]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)
train_labels= train['TARGET']
train, test = train.align(test, join = 'inner', axis = 1)
train['TARGET'] = train_labels

In [6]:
test = test.drop('TARGET', axis = 1)

In [7]:
# Devide train data into 3: train A, B, and C
from sklearn.model_selection import train_test_split
train_A, train_B = train_test_split(train, test_size = 0.6, random_state = 3)
train_B, train_C = train_test_split(train_B, test_size = 0.3, random_state = 3)


In [8]:
print(train_A.shape, train_B.shape, train_C.shape, test.shape)

(123004, 1157) (129154, 1157) (55353, 1157) (48744, 1156)


## Models

##### We are using Adaboost, GradientBoost, xgboost RandomForest, LogRes, KNN, SVM
##### ^low depth, medium depth, high depth, medium size

In [10]:
# First level models to use
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [13]:
ada = AdaBoostClassifier(n_estimators = 100, learning_rate = 0.01) # AdaBoost
gbc = GradientBoostingClassifier(learning_rate = 0.01,
                                 n_estimators = 600,
                                 max_depth = 10, # low depth
                                 subsample = 0.5) # Gradient Boosting Classifier
rf_1 = RandomForestClassifier(n_estimators = 100, max_depth = 75, # medium depth
                             n_jobs = -1)# Random Forest
rf_2 = RandomForestClassifier(n_estimators = 750, max_depth = 200,
                             n_jobs = -1) # high depth
log = LogisticRegression(C = 0.1, n_jobs = -1) # Logistic Regression
knn = KNeighborsClassifier(n_neighbors = 2, n_jobs = -1) # K Neighbors Classifier
svc = SVC(C=0.2, kernel='poly', degree=2, gamma='auto', 
          coef0=0.0, shrinking=True, probability=False, 
          tol=0.001, cache_size=200, class_weight=None, 
          verbose=True, max_iter=-1, decision_function_shape='ovr', 
          random_state=None)

In [14]:
from sklearn.preprocessing import MinMaxScaler, Imputer

In [15]:
def train_first_layer_model(clf, train_data):
    train_labels = train_data['TARGET']
    train_features = train_data.drop('TARGET', axis = 1)
    #encoded_train_features = pd.get_dummies(train_features)

    imputer = Imputer(strategy = 'median')
    filled_train_features = imputer.fit_transform(train_features)
    scaler = MinMaxScaler(feature_range = (0, 1))
    scaled_train_features = scaler.fit_transform(filled_train_features)
    
    clf.fit(scaled_train_features, train_labels)
    
    return clf

In [16]:
# Train all the first level models

In [16]:
ada = train_first_layer_model(ada, train_A) #AdaBoost

In [17]:
gbc = train_first_layer_model(gbc, train_A) #GradientBoostingClassifier

In [18]:
rf_1= train_first_layer_model(rf_1, train_A) #RandomForest 1

In [19]:
rf_2= train_first_layer_model(rf_2, train_A) #RandomForest 2

In [20]:
log = train_first_layer_model(log, train_A) #log

C:\Users\JoonH\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


In [21]:
knn = train_first_layer_model(knn, train_A) #knn

In [22]:
svc = train_first_layer_model(svc, train_A) #svm

[LibSVM]

In [23]:
# make predictions of train_B, train_C, and test with first layer models
# build a meta model and train it with B, and validate with C for hypter parameter tuning
# train meta model on A,B,C and predict test

def process_data(data):
    
    if 'TARGET' in data.columns:
        data_features = data.drop('TARGET', axis = 1)
    else:
        data_features = data.copy()
        
    #encoded_data_features = pd.get_dummies(data_features)
    imputer = Imputer(strategy = 'median')
    filled_data_features = imputer.fit_transform(data_features)
    scaler = MinMaxScaler(feature_range = (0, 10))
    scaled_data_features = scaler.fit_transform(filled_data_features)
    
    return scaled_data_features

def predict_first_layer(clf, dataB, dataC, test):
    B_features = process_data(dataB)
    C_features = process_data(dataC)
    print(B_features.shape, C_features.shape)
    #train_features = process_data(dataB.append(dataC))
    #B_features = train_features[:len(dataB)]
    #C_features = train_features[len(dataB):]
    test_features = process_data(test)
    B_meta = clf.predict(B_features)
    C_meta = clf.predict(C_features)
    test_meta = clf.predict(test_features)
    
    return B_meta, C_meta, test_meta


In [24]:
ada_B_meta, ada_C_meta, ada_test_meta = predict_first_layer(ada, train_B, 
                                                            train_C, test)

(129154, 1156) (55353, 1156)


In [25]:
gbc_B_meta, gbc_C_meta, gbc_test_meta = predict_first_layer(gbc, train_B, 
                                                            train_C, test)

(129154, 1156) (55353, 1156)


In [26]:
rf_1_B_meta, rf_1_C_meta, rf_1_test_meta = predict_first_layer(rf_1, train_B, 
                                                               train_C, test)

(129154, 1156) (55353, 1156)


In [27]:
rf_2_B_meta, rf_2_C_meta, rf_2_test_meta = predict_first_layer(rf_2, train_B, 
                                                               train_C, test)

(129154, 1156) (55353, 1156)


In [28]:
log_B_meta, log_C_meta, log_test_meta = predict_first_layer(log, train_B, 
                                                            train_C, test)

(129154, 1156) (55353, 1156)


In [29]:
knn_B_meta, knn_C_meta, knn_test_meta = predict_first_layer(knn, train_B, 
                                                            train_C, test)

(129154, 1156) (55353, 1156)


In [30]:
svc_B_meta, svc_C_meta, svc_test_meta = predict_first_layer(svc, train_B, 
                                                            train_C, test)

(129154, 1156) (55353, 1156)


In [31]:
# Create metadata for meta_B
# Note that in our dataframe we will add '_eng' to our column names
# to indicate they used the data that underwent feature engineering.

ada_B_meta = pd.DataFrame(ada_B_meta, columns = ['ada_eng'])
gbc_B_meta = pd.DataFrame(gbc_B_meta, columns = ['gbc_eng'])
rf_1_B_meta = pd.DataFrame(rf_1_B_meta, columns = ['rf_1_eng'])
rf_2_B_meta = pd.DataFrame(rf_2_B_meta, columns = ['rf_2_eng'])
log_B_meta = pd.DataFrame(log_B_meta, columns = ['log_eng'])
knn_B_meta = pd.DataFrame(knn_B_meta, columns = ['knn_eng'])
svc_B_meta = pd.DataFrame(svc_B_meta, columns = ['svc_eng'])

In [33]:
# meta_C
ada_C_meta = pd.DataFrame(ada_C_meta, columns = ['ada_eng'])
gbc_C_meta = pd.DataFrame(gbc_C_meta, columns = ['gbc_eng'])
rf_1_C_meta = pd.DataFrame(rf_1_C_meta, columns = ['rf_1_eng'])
rf_2_C_meta = pd.DataFrame(rf_2_C_meta, columns = ['rf_2_eng'])
log_C_meta = pd.DataFrame(log_C_meta, columns = ['log_eng'])
knn_C_meta = pd.DataFrame(knn_C_meta, columns = ['knn_eng'])
svc_C_meta = pd.DataFrame(svc_C_meta, columns = ['svc_eng'])

In [34]:
# test_meta
ada_test_meta = pd.DataFrame(ada_test_meta, columns = ['ada_eng'])
gbc_test_meta = pd.DataFrame(gbc_test_meta, columns = ['gbc_eng'])
rf_1_test_meta = pd.DataFrame(rf_1_test_meta, columns = ['rf_1_eng'])
rf_2_test_meta = pd.DataFrame(rf_2_test_meta, columns = ['rf_2_eng'])
log_test_meta = pd.DataFrame(log_test_meta, columns = ['log_eng'])
knn_test_meta = pd.DataFrame(knn_test_meta, columns = ['knn_eng'])
svc_test_meta = pd.DataFrame(svc_test_meta, columns = ['svc_eng'])

In [35]:
# forming dataframes
B_meta_data = pd.concat([ada_B_meta, gbc_B_meta, rf_1_B_meta, rf_2_B_meta,
                        log_B_meta, knn_B_meta, svc_B_meta],axis = 1)

C_meta_data = pd.concat([ada_C_meta, gbc_C_meta, rf_1_C_meta, rf_2_C_meta,
                        log_C_meta, knn_C_meta, svc_C_meta],axis = 1)

test_meta_data = pd.concat([ada_test_meta, gbc_test_meta, rf_1_test_meta, rf_2_test_meta,
                           log_test_meta, knn_test_meta, svc_test_meta],axis = 1)

In [36]:
B_meta_data.to_csv("eng_data_B_meta.csv")
C_meta_data.to_csv("eng_data_C_meta.csv")
test_meta_data.to_csv("eng_data_test_meta.csv")

In [393]:
features = pd.concat([B_meta_data.copy(), labels],axis = 1)
test_features = test_meta_data.copy()

In [396]:
# thank you Will Koehrsen for an amazing kernel / this method!

def model(features, test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    #train_ids = features['SK_ID_CURR']
    #test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    #features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    features = features.drop(columns = ['TARGET'])
    #test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=1000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.01, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    #submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    submission = pd.DataFrame({'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [397]:
submission, fi, metrics = model(features, test_features)
print('Baseline metrics')
print(metrics)

Training Data Shape:  (92253, 9)
Testing Data Shape:  (48744, 9)
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[67]	valid's auc: 0.526473	train's auc: 0.520415
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[98]	valid's auc: 0.51951	train's auc: 0.522189
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.522222	train's auc: 0.521511
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.51921	train's auc: 0.522252
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[68]	valid's auc: 0.520697	train's auc: 0.521894
Baseline metrics
      fold     train     valid
0        0  0.520415  0.526473
1        1  0.522189  0.519510
2        2  0.521511  0.522222
3        3  0.522252  0.519210
4        4  0.521894  0.520697
5

In [398]:
submission.to_csv('credit_meta_baseline.csv', index = False)

#### Plan:
##### 1. Break down training into A,B,C
##### 2. Build numerous various models (fitted to A) and form predictions for B,C.test
##### 3. Build a meta_model (lightgbm) that trains on B_meta and B_true_label
##### 4. Validate model on C_meta and C_true_label
##### 5. Make predictions on test with meta_model via test_meta